In [16]:
import neo4j as neo

In [17]:
url = 'bolt://localhost:7687'
auth = ('neo4j', 'neo4j_')

In [18]:
def _run_query(tx, query, **args):
    """
    Run one query statement in transaction, and return result
    """
    result = tx.run(query, **args)
    return result
    
def _run_transaction(statement_list):
    # Make neo4j driver object
    driver = neo.GraphDatabase.driver(url, auth=auth)
    try:
        # Start neo4j session
        with driver.session() as session:
            for statement in statement_list:
                query = statement['query']
                args = statement.get('args', {})
                # Put each statement into transaction
                result = session.write_transaction(_run_query, query, **args)
                print(result.values())
                print([node for node in result.graph().nodes])
                print([r for r in result.graph().relationships])
    finally:
        driver.close()

In [19]:
"""
Create or merge object in neo4j
"""

statements = [
    {
        'query': 'MERGE (person:Person {id: {id}, name: {name}, gender: {gender}}) '
                 'RETURN person',
        'args': {
            'id': 1,
            'name': 'Alvin',
            'gender': 'M' 
        }
    },
    {
        'query': 'MERGE (person:Person {id: {id}, name: {name}, gender: {gender}}) '
                 'RETURN person',
        'args': {
            'id': 2,
            'name': 'Emma',
            'gender': 'F' 
        }
    }
]

_run_transaction(statements)

[[<Node id=1 labels={'Person'} properties={'gender': 'M', 'name': 'Alvin', 'id': 1}>]]
[<Node id=1 labels={'Person'} properties={'gender': 'M', 'name': 'Alvin', 'id': 1}>]
[]
[[<Node id=2 labels={'Person'} properties={'gender': 'F', 'name': 'Emma', 'id': 2}>]]
[<Node id=2 labels={'Person'} properties={'gender': 'F', 'name': 'Emma', 'id': 2}>]
[]


In [20]:
"""
Update exist object
"""

statements = [
    {
        'query': 'MATCH (person:Person)'
                 'WHERE person.id = {id} '
                 'SET person.birthday = {birthday} '
                 'RETURN person',
        'args': {
            'id': 1,
            'birthday': '1981-03-17'
        }
    },
    {
        'query': 'MATCH (person:Person)'
                 'WHERE person.id = {id} '
                 'SET person.birthday = {birthday} '
                 'RETURN person',
        'args': {
            'id': 2,
            'birthday': '1985-03-29'
        }
    }
]

_run_transaction(statements)

[[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>]]
[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>]
[]
[[<Node id=2 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]]
[<Node id=2 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[]


In [21]:
"""
Build relationship
"""

statements = [
    {
        'query': 'MATCH (p1:Person), (p2:Person)'
                 'WHERE p1.id = {id1} AND p2.id = {id2} '
                 'MERGE (p1)-[r:Married]->(p2) '
                 'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        }
    },
    {
        'query': 'MATCH (p1:Person), (p2:Person) '
                 'WHERE p1.id = {id1} AND p2.id = {id2} '
                 'MERGE (p2)-[r:Married]->(p1) '
                 'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        }
    }
]

_run_transaction(statements)

[[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=2 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>, 'Married']]
[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=2 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[]
[[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=2 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>, 'Married']]
[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=2 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[]


In [22]:
"""
Create object and build relationship
"""

statements = [
    {
        'query': 'MATCH (person:Person) '
                 'WHERE person.id = {person_id} '
                 'MERGE (cat:Cat { id: {cat_id}, name: {cat_name} }) '
                 'MERGE (person)-[r:Feed]->(cat) '
                 'RETURN person, cat, type(r)',
        'args': {
            'person_id': 1,
            'cat_id': 3,
            'cat_name': "Lily"
        }
    }
]

_run_transaction(statements)

[[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=3 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>, 'Feed']]
[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=3 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>]
[]


In [23]:
"""
Find person who feed cat Lily
"""

statements = [
    {
        'query': 'MATCH (person:Person)-[r:Feed]->(cat:Cat) '
                 'WHERE cat.name = {cat_name} '
                 'RETURN person, cat, type(r)',
        'args': {
            'cat_name': "Lily"
        }
    }
]

_run_transaction(statements)

[[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=3 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>, 'Feed']]
[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=3 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>]
[]


In [59]:
"""
Find persons who has relationship with cat Lily
"""

statements = [
    {
        'query': 'MATCH (person:Person)-[r*]->(cat:Cat) '
                 'WHERE cat.name = {cat_name} '
                 'RETURN DISTINCT person.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

_run_transaction(statements)

[['Alvin'], ['Emma']]
[]
[]


In [61]:
"""
Find who has relationship with cat Lily
"""

statements = [
    {
        'query': 'MATCH (p1:Person)-[:Married]-(p2:Person)-[r:Feed*]->(c:Cat) '
                 'WHERE c.name={cat_name} '
                 'RETURN DISTINCT p1.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

_run_transaction(statements)

[['Emma']]
[]
[]


In [67]:
"""
Find graph to cat Lily
"""

statements = [
    {
        'query': 'MATCH g = (person:Person)-[r*]->(cat:Cat) '
                 'WHERE cat.name = {cat_name} '
                 'RETURN g, length(g)',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

_run_transaction(statements)

[[<Path start=<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> end=<Node id=3 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=1>, 1], [<Path start=<Node id=2 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}> end=<Node id=3 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=2>, 2], [<Path start=<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> end=<Node id=3 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=3>, 3]]
[<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=3 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>, <Node id=2 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[<Relationship id=3 nodes=(<Node id=1 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alv

In [79]:
"""
Find last children node in all relationship
"""

statements = [
    {
        'query': 'MATCH ()-[*]->(n) '
                 'WHERE NOT (n)-[]->() '
                 'RETURN DISTINCT n.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

_run_transaction(statements)

[['Lily']]
[]
[]


In [87]:
"""
Delete nodes with relationships
"""

statements = [
    {
        'query': 'MATCH ()-[mr:Married]->() '
                 'MATCH ()-[fr:Feed]->() '
                 'OPTIONAL MATCH (ps:Person) '
                 'OPTIONAL MATCH(cs:Cat) '
                 'DELETE mr, fr, ps, cs '
                 'RETURN mr, fr, ps, cs'
    }
]

_run_transaction(statements)

[]
[]
[]


In [88]:
"""
Import from csv
"""

statements = [
    {
        'query': 'LOAD CSV WITH HEADERS FROM "file:///data.csv" AS line '
                 'MERGE (p1:Person {id: line["id"], name: line["name"]}) '
                 'MERGE (p2:Person {id: line["married"]}) '
                 'MERGE (p1)-[:Married]->(p2) '
                 'RETURN p1, p2'
    }
]

_run_transaction(statements)

[[<Node id=21 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>, <Node id=1 labels={'Person'} properties={'name': 'Emma', 'id': '2'}>], [<Node id=1 labels={'Person'} properties={'name': 'Emma', 'id': '2'}>, <Node id=21 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>]]
[<Node id=21 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>, <Node id=1 labels={'Person'} properties={'name': 'Emma', 'id': '2'}>]
[]
